In [1]:
import numpy as np
import pandas as pd


In [2]:
wines_df = pd.read_csv('./data/production/wine_production.csv',  index_col='Unnamed: 0')
wines_df.head()


,aroma,weight,sweet,acid,salt,piquant,fat,bitter,flavor
"Bordeaux-style Red Blend Bourg & Blaye, Bordeaux, France",[ 3.58181261e-02 -1.34934425e-01 1.22975700e-...,0.755563,0.000000,0.516379,0.203458,0.394980,0.447658,0.647174,0.962639
"Bordeaux-style Red Blend Coastal Region, South Africa",[ 1.01381801e-01 -2.23976374e-01 9.08020735e-...,0.376757,0.517928,0.363943,0.203459,0.423887,0.391066,0.533777,0.435152
"Bordeaux-style Red Blend Colchagua Valley, Central Valley, Chile",[ 7.88071379e-02 -2.10951209e-01 5.93466125e-...,0.508481,0.395319,0.265614,0.203459,0.095043,0.275438,0.350044,0.276377
"Bordeaux-style Red Blend Columbia Valley, Washington, USA",[ 6.91195950e-02 -2.09454700e-01 1.14624642e-...,0.567349,0.433776,0.359936,0.200733,0.370159,0.445596,0.461552,0.410747
"Bordeaux-style Red Blend Entre-Deux-Mers, Bordeaux",[ 4.58370000e-02 -1.44405901e-01 9.85255539e-...,0.787105,0.030708,0.426893,0.203459,0.401985,0.450556,0.649967,1.000000
